In [2]:
import pandas as pd
import numpy as np
import torch
import os
import shutil
import sys
from PIL import Image
import utility_functions as uf
import subprocess

# testing area

In [3]:
#check is there any slide not processed by rightside selecting yet, and print the number of rightside patches in each slide 
directory_path = '/home/exon_storage1/aml_slide/patches/'
count = 0
for file in os.listdir(directory_path):
    if  os.path.exists(directory_path+file+"/rightside_patch"):
        count += 1
        patches_path = directory_path+file+"/rightside_patch/"
        number_of_patches = len(os.listdir(patches_path))
        print( file+" : "+str(len(os.listdir(patches_path))))
    else:
        print(file+" is not processed yet")
print("there are {} slides processed".format(count))

A347 : 14770
A330 : 19455
A1151 : 11137
A799 : 20970
A777 : 6623
A607 : 18738
A670 : 19646
A1056 : 12793
A490 : 17101
A824 : 16777
A409 : 21543
A143 : 8712
A134 : 19503
A22 : 22707
A149 : 22709
A28 : 10296
A579 : 17425
A403 : 22793
A597 : 21651
A474 : 9026
A1114 : 13914
A1080 : 20543
A648 : 12132
A732 : 13903
A1019 : 9448
A745 : 8617
A205 : 21828
A308 : 3586
A642 : 25685
A1169 : 18409
A635 : 26048
A738 : 19512
A1013 : 20401
A106 : 26152
A171 : 23421
A536 : 21223
A982 : 18887
A816 : 16436
A966 : 8314
A446 : 9581
A988 : 21551
A431 : 23395
A224 : 9087
A1032 : 21538
A719 : 21528
A614 : 22047
A764 : 7716
A1038 : 10702
A1142 : 19586
A669 : 12542
A1135 : 17504
A323 : 21990
A410 : 6911
A467 : 18005
A1236 : 20308
A150 : 23526
A837 : 23571
A560 : 20707
A651 : 26804
A1099 : 21126
A626 : 18108
A382 : 5811
A311 : 7618
A366 : 22177
A721 : 17287
A756 : 16740
A1093 : 5616
A1170 : 16023
A168 : 7912
A455 : 17108
A422 : 16324
A902 : 13732
A896 : 11380
A872 : 21765
A428 : 22739
A991 : 5836
A525 : 19285
A1

In [ ]:
#this is a script to remove the slides with the suffix of x or Rx, and rename the slides with the suffix of Lx
path = "/home/exon_storage1/aml_slide/patches/"
for file in os.listdir(path):
    detector = file.split("_")[-1]
    detector = detector.split(".")[0]
    if detector =="Lx":
        oldname = path+file
        newname = oldname.replace("_Lx","")
        print(oldname)
        os.rename(oldname, newname)
    elif detector == "x" or detector == "Rx":
        print(path+file)
        # os.remove(path+file)
        shutil.rmtree(path+file)

In [ ]:
#this script is used to normalize the patches and save them in a new folder, with a blurry filter
sys.path.insert(1, '/home/weber50432/AML_image_processing/HEnorm_python/')
from normalizeStaining import normalizeStaining , is_blurry
directory_path = '/home/weber50432/AML_image_processing/PyHIST/output/'
for file in os.listdir(directory_path):
    if  os.path.exists(directory_path+file+"/rightside_patch"):
        patches_path = directory_path+file+"/rightside_patch/"
        if os.path.exists(directory_path+file+"/rightside_patch_norm"):
            print(file + " rightside_patch_norm is already created !")
            continue
        else:
            os.mkdir(directory_path+file+"/rightside_patch_norm/")
        if os.path.exists(directory_path+file+"/rightside_patch_blurry"):
            print(file + " rightside_patch_blurry is already created !")
        else:
            os.mkdir(directory_path+file+"/rightside_patch_blurry/")
        patches_path_blurry = directory_path+file+"/rightside_patch_blurry/"
        patches_path_norm = directory_path+file+"/rightside_patch_norm/"
        for img in os.listdir(patches_path):
            if img.endswith(".png"):
                img_num = str(img)
                img_path = patches_path+img
                img = np.array(Image.open(img_path))
                Inorm, H, E = normalizeStaining(img = img,
                      saveFile = None,
                      Io = 240,
                      alpha = 1,
                      beta = 0.15)
                if is_blurry(Inorm):
                    Image.fromarray(Inorm).save(patches_path_blurry+img_num)
                    # os.remove(img_path)
                    print(img_num+" is blurry !")
                else:
                    Image.fromarray(Inorm).save(patches_path_norm+img_num)
                    print(img_num+" is saved !")

In [ ]:
def get_pid(process_name):
    p = subprocess.Popen(['pidof', process_name], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    pid, error = p.communicate()
    return int(pid.split()[0])

# Example usage
pid = get_pid('python')
print(pid)

In [19]:
# find Lx slides
path = "/home/exon_storage1/aml_slide/patches/"
for file in os.listdir(path):
    if  os.path.exists(directory_path+file+"/rightside_patch"):
        patches_path = directory_path+file+"/rightside_patch/"
        if os.listdir(patches_path)[0].split("_")[1] == "Lx":
            print(os.listdir(patches_path)[0].split("_")[0])

A153_Lx_51786.png
A1197_Lx_35627.png
A201_Lx_21614.png
A466_Lx_24697.png
A152_Lx_66094.png


In [14]:
#check the input data for ROI region selecting
output_path = "/home/weber50432/AML_image_processing/HCT_region_select/output/"
slide_path = "/home/exon_storage1/aml_slide/patches/"
exist_slide_list = os.listdir(output_path)
slide_list =[]
for i in os.listdir(slide_path):
    if not i in exist_slide_list:
        slide_list.append(i)
slide_list.sort(
    key=lambda x: int(x[1:]) if x[1:].isdigit() else 0
)
for i in slide_list:
  input_path = "{}{}/rightside_patch".format(slide_path,i)
  if not os.path.exists(input_path):
     print("No rightside_patch folder for slide {}".format(i))

No rightside_patch folder for slide A1306


In [9]:
saved_path = '/home/exon_storage1/aml_slide/patches/'
slide_path = '/home/exon_storage1/aml_slide/renameByUPN/'
slide_list = []
for file in os.listdir(slide_path):
    if file.endswith('.ndpi'):
        file_num = file.split(".")[0]
        if not file_num in os.listdir(saved_path):
            slide_list.append(slide_path+file)
print(len(slide_list))
for i in slide_list:
    print(i)
    # os.system('python pyhist.py --content-threshold 0.05 --output {} --output-downsample 1 --save-patches --save-tilecrossed-image --info "verbose" {}'.format(saved_path,i))

40
/home/exon_storage1/aml_slide/renameByUPN/A1306.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A542.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A308.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A792.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A777.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A1327.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A362.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A143.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A776.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A124.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A396.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A25.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A1307.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A311.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A292.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A59.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A1315.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A759.ndpi
/home/exon_storage1/aml_slide/renameByUPN/A764.ndpi
/home/e